In [1]:
# Import relevant libaries
%matplotlib qt5 

import mne
import os
from mne.preprocessing import ICA
from mne.preprocessing import create_eog_epochs, create_ecg_epochs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
vhdr_files = ['382733.vhdr', '382733_2.vhdr', '477819.vhdr', '477819_2.vhdr', '554432.vhdr', '554432_2.vhdr', '884723.vhdr', '884723_2.vhdr', '928376.vhdr', '928376_2.vhdr', '990291.vhdr', '990291_2.vhdr']
print(vhdr_files)
temp_list = vhdr_files[0:2]
print(temp_list)

['382733.vhdr', '382733_2.vhdr', '477819.vhdr', '477819_2.vhdr', '554432.vhdr', '554432_2.vhdr', '884723.vhdr', '884723_2.vhdr', '928376.vhdr', '928376_2.vhdr', '990291.vhdr', '990291_2.vhdr']
['382733.vhdr', '382733_2.vhdr']


In [3]:
filename = temp_list[0]
print(filename)

if filename[6] == '.':
    epochs_EN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:6] + '-epo.fif')
else:
    epochs_EN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:8] + '-epo.fif')
print(epochs_EN)

382733.vhdr
Reading data/epochs/processed_epochs/382733-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     700.00 ms
        0 CTF compensation matrices available
120 matching events found
Applying baseline correction (mode: mean)
120 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
<EpochsFIF  |   120 events (all good), -0.2 - 0.7 sec, baseline [-0.2, 0], ~26.5 MB, data loaded,
 'Dislike Conf': 2
 'Favourite Conf': 1
 'Favourite Match': 1
 'Finnish Conf': 24
 'Finnish Match': 25
 'Foreing Conf': 22
 'Foreing Match': 27
 'Neutrals': 10
 'Unknowns': 8>


In [4]:
filename = temp_list[1]
print(filename)

if filename[6] == '.':
    epochs_FIN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:6] + '-epo.fif')
else:
    epochs_FIN = mne.read_epochs('data/epochs/processed_epochs/' + filename[:8] + '-epo.fif')
print(epochs_FIN)

382733_2.vhdr
Reading data/epochs/processed_epochs/382733_2-epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     700.00 ms
        0 CTF compensation matrices available
120 matching events found
Applying baseline correction (mode: mean)
120 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
<EpochsFIF  |   120 events (all good), -0.2 - 0.7 sec, baseline [-0.2, 0], ~26.5 MB, data loaded,
 'Dislike Conf': 1
 'Dislike Match': 1
 'Favourite Conf': 1
 'Favourite Match': 1
 'Finnish Conf': 23
 'Finnish Match': 27
 'Foreing Conf': 25
 'Foreing Match': 23
 'Neutrals': 10
 'Unknowns': 8>


In [5]:
# Lets study Finnish Match / Conf and Foreing Match / Conf
finmatc = epochs_EN['Finnish Match'].average()
finconf = epochs_EN['Finnish Conf'].average()
formatc = epochs_EN['Foreing Match'].average()
forconf = epochs_EN['Foreing Conf'].average()

all_evokeds_EN = [finmatc, finconf, formatc, forconf]

finmatc = epochs_EN['Finnish Match']
finconf = epochs_EN['Finnish Conf']
formatc = epochs_EN['Foreing Match']
forconf = epochs_EN['Foreing Conf']
all_epochs_EN = [finmatc, finconf, formatc, forconf]
print(all_evokeds_EN)

# Lets study Finnish Match / Conf and Foreing Match / Conf
finmatc = epochs_FIN['Finnish Match'].average()
finconf = epochs_FIN['Finnish Conf'].average()
formatc = epochs_FIN['Foreing Match'].average()
forconf = epochs_FIN['Foreing Conf'].average()

all_evokeds_FIN = [finmatc, finconf, formatc, forconf]

finmatc = epochs_FIN['Finnish Match']
finconf = epochs_FIN['Finnish Conf']
formatc = epochs_FIN['Foreing Match']
forconf = epochs_FIN['Foreing Conf']
all_epochs_FIN = [finmatc, finconf, formatc, forconf]
print(all_evokeds_FIN)

[<Evoked  |  'Finnish Match' (mean, N=25), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Finnish Conf' (mean, N=24), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Match' (mean, N=27), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Conf' (mean, N=22), [-0.2, 0.7] sec, 32 ch, ~311 kB>]
[<Evoked  |  'Finnish Match' (mean, N=27), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Finnish Conf' (mean, N=23), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Match' (mean, N=23), [-0.2, 0.7] sec, 32 ch, ~311 kB>, <Evoked  |  'Foreing Conf' (mean, N=25), [-0.2, 0.7] sec, 32 ch, ~311 kB>]


In [42]:
# FINNISH MATCH VS ENGLISH MATCH?

suomeksi_FINMAT_gfp = np.sum(all_evokeds_FIN[0].data ** 2, axis=0)
#suomeksi_FINMAT_gfp = mne.baseline.rescale(suomeksi_FINMAT_gfp, all_evokeds_FIN[0].times * 1e3, baseline=(None, 0))

suomeksi_FORMAT_gfp = np.sum(all_evokeds_FIN[2].data ** 2, axis=0)
#englanniksi_FINMAT_gfp = mne.baseline.rescale(englanniksi_FINMAT_gfp, all_evokeds_FIN[0].times * 1e3, baseline=(None, 0))

In [43]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
from sklearn.utils import resample

combined = np.concatenate((suomeksi_FINMAT_gfp, suomeksi_FORMAT_gfp), axis=0)

perms_FMAT = []
perms_EMAT = []

for i in range(10000):
    np.random.seed(i)
    perms_FMAT.append(resample(combined, n_samples = len(suomeksi_FINMAT_gfp)))
    perms_EMAT.append(resample(combined, n_samples = len(suomeksi_FORMAT_gfp)))
    
dif_bootstrap_means = (np.mean(perms_FMAT, axis=1)-np.mean(perms_EMAT, axis=1))

fig = plt.figure(figsize=(10,3))
ax = plt.hist(dif_bootstrap_means, bins=40)

plt.xlabel('Mean difference in terms of GFP')
plt.ylabel('Frequency')
plt.title('Bootstrapped Population (Combined data)')
plt.show()

In [44]:
plt.plot(all_evokeds_FIN[0].times * 1e3, suomeksi_FINMAT_gfp)
plt.plot(all_evokeds_FIN[2].times * 1e3, suomeksi_FORMAT_gfp)
plt.show()

In [40]:
obs_difs = (np.mean(suomeksi_FINMAT_gfp) - np.mean(suomeksi_FORMAT_gfp))
print('observed difference in means: {}'.format(obs_difs))

p_value = dif_bootstrap_means[dif_bootstrap_means >= obs_difs].shape[0]/10000
print('p-value: {}'.format(p_value))

observed difference in means: -3.9738766911620236e-10
p-value: 1.0


In [41]:
FMAT_gfp_bootstrap = []
for i in range(10000):
    np.random.seed(i)
    FMAT_gfp_bootstrap.append((resample(suomeksi_FINMAT_gfp)))

FMAT_gfp_bootstrap = np.mean(FMAT_gfp_bootstrap, axis=1)
plt.hist(FMAT_gfp_bootstrap)

EMAT_gfp_bootstrap = []
for i in range(10000):
    np.random.seed(i)
    EMAT_gfp_bootstrap.append((resample(englanniksi_FINMAT_gfp)))

EMAT_gfp_bootstrap = np.mean(EMAT_gfp_bootstrap, axis=1)
plt.hist(EMAT_gfp_bootstrap)

(array([2.000e+00, 2.100e+01, 1.880e+02, 9.340e+02, 2.444e+03, 3.147e+03,
        2.296e+03, 7.740e+02, 1.730e+02, 2.100e+01]),
 array([7.56049295e-10, 7.77888406e-10, 7.99727517e-10, 8.21566628e-10,
        8.43405739e-10, 8.65244850e-10, 8.87083961e-10, 9.08923072e-10,
        9.30762184e-10, 9.52601295e-10, 9.74440406e-10]),
 <a list of 10 Patch objects>)